In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd

import utils_gen as u

/usr/local/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [3]:
conn = u.load_connection()

First, look at the posts that we want to remove and determine what the impact will be to the 2018 rb count.

In [4]:
df_remove = pd.read_csv('no_signup_posts.csv')

In [5]:
remove_posts = u.sql_stringify_list(df_remove['post_id'].values)

In [6]:
sqlremove = '''select * from campaign_activity where post_id in ({})'''.format(remove_posts)
df_ca_remove = pd.read_sql(sqlremove, conn)

In [7]:
df_ca_remove['post_attribution_date'].value_counts()

2010-01-30 13:06:00    664
2010-02-15 22:25:00    437
2018-02-26 20:19:41     51
2018-03-27 22:47:38      5
Name: post_attribution_date, dtype: int64

Most look like they were already being counted in 2010, so the total impact to 2018 will be -56 rbs.

Now, let's look at what the impact will be once we correctly attribute posts to their signups. This requires confirming that those signup ids do not have any other social shares that would lead to double counting.

In [8]:
df_correct = pd.read_csv('fb_post_reattribution_ids.csv')

In [9]:
correct_posts = u.sql_stringify_list(df_correct['post_id'].values)
correct_signups = u.sql_int_list(df_correct['signup_id'].values)

In [10]:
sql_signups = '''select * from campaign_activity where signup_id in ({})'''.format(correct_signups)
df_signups = pd.read_sql(sql_signups, conn)

In [11]:
df_signups['post_class'].value_counts()

voter-reg - may-2018-turbovote       445
photo - default                      205
text - default                        27
voter-reg - june-2018-turbovote       26
voter-reg - july-2018-turbovote        9
voter-reg - april-2018-turbovote       1
voter-reg - june-2014-rockthevote      1
Name: post_class, dtype: int64

Looks like there are records that don't have any posts associated with them. Let's see how many unique signup_ids don't have any posts

In [12]:
null_signup_ids = set(df_signups[df_signups['post_class'].isnull()]['signup_id'].unique())
not_null_signup_ids = set(df_signups[df_signups['post_class'].notnull()]['signup_id'].unique())

In [13]:
len(null_signup_ids & not_null_signup_ids)

0

In [14]:
len(df_signups['signup_id'].unique()), len(null_signup_ids), len(not_null_signup_ids)

(4498, 3922, 576)

That's safe, so we know for sure that 3,923 of the 4,498 don't have any posts associated with them. Let's double check the 576.

In [15]:
df_signups[df_signups['post_class'].notnull()]['post_class'].value_counts()

voter-reg - may-2018-turbovote       445
photo - default                      205
text - default                        27
voter-reg - june-2018-turbovote       26
voter-reg - july-2018-turbovote        9
voter-reg - april-2018-turbovote       1
voter-reg - june-2014-rockthevote      1
Name: post_class, dtype: int64

None of these signup ids have social shares either, so all posts that need to be updated are the only social shares for their correct signup id. Final step is to look at impact for 2018.

In [16]:
sql_posts = '''select * from campaign_activity where post_id in ({})'''.format(correct_posts)

In [17]:
df_posts = pd.read_sql(sql_posts, conn)

In [18]:
df_posts['post_attribution_date'].value_counts()

2010-02-15 22:25:00    3510
2010-01-30 13:06:00     972
2018-02-26 20:19:41     379
2018-03-27 22:47:38      63
2018-05-03 21:35:30      37
2018-05-02 18:46:03       4
2018-05-31 17:18:52       3
2018-05-11 15:29:52       2
2018-07-01 04:55:13       1
2018-05-30 19:30:10       1
2018-05-28 12:07:13       1
Name: post_attribution_date, dtype: int64

There will be some redistribution of these once they're using post created at dates, but from above will add 4482 rbs to 2018 totals (not including -56 from the first section or other cleanup from updated deduping).